In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.data.preprocessing import prepare_datasets


In [2]:
prepare_datasets(
    input_path="../data/raw/events.csv",
    output_dir="../data/processed",
    min_user_events=3,
    min_item_events=5,
)


In [3]:
import pandas as pd

train = pd.read_csv("../data/processed/train.csv")
val = pd.read_csv("../data/processed/val.csv")
test = pd.read_csv("../data/processed/test.csv")

len(train), len(val), len(test)


(1008982, 126123, 126123)

In [4]:
import pandas as pd

train = pd.read_csv("../data/processed/train.csv")
val = pd.read_csv("../data/processed/val.csv")
test = pd.read_csv("../data/processed/test.csv")

len(train), len(val), len(test)


(1008982, 126123, 126123)

In [5]:
train.groupby("visitorid").size().min(), train.groupby("itemid").size().min()


(np.int64(1), np.int64(1))

In [6]:
import pandas as pd

train = pd.read_csv("../data/processed/train.csv")
val = pd.read_csv("../data/processed/val.csv")
test = pd.read_csv("../data/processed/test.csv")

events_all = pd.concat([train, val, test], ignore_index=True)

user_activity_all = events_all.groupby("visitorid").size()
item_pop_all = events_all.groupby("itemid").size()

user_activity_all.min(), item_pop_all.min()


(np.int64(1), np.int64(1))

### Важное замечание о фильтрации и временном сплите

Фильтрация пользователей и товаров применяется до временного разбиения данных.
После split-а (train / validation / test) отдельные сплиты и даже их объединение
могут содержать пользователей и товары с меньшим числом событий,
так как часть их взаимодействий остаётся только в одном временном интервале.

Это является ожидаемым эффектом упрощённого preprocessing-пайплайна
и не препятствует построению baseline-моделей рекомендаций.

В дальнейшем пайплайн может быть улучшен путём:
- выполнения временного split-а перед фильтрацией,
- расчёта порогов `min_user_events` и `min_item_events` только по train-части,
- применения полученных фильтров к validation и test.
